### Imports

In [ ]:
%pip install -r dependencies.txt

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

### CONSTANTS

In [ ]:
ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

In [ ]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

### Read data

In [ ]:
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", "relvol_nt_0"]
train_df = helper.get_df(TRAIN_START_DATE, TRAIN_END_DATE, x_cols)

### Interaction Terms

In [ ]:
# interactingTerms = [["relvol_nt_0", col] for col in x_cols[:-1]]
# interactingTerms

In [ ]:
# interactingTerms_df = helper.get_df_with_interaction_terms(train_df, interactingTerms)

In [ ]:
# interactingTerms_df.head()

In [ ]:
# rosy: making my own interacting terms

# col_pairs = [
#     ['relvol_nt_0', 'rrirpnxm_nt_0'],
#     ["relvol_lst15_0", "rrirpnxm_lst15_0"],
#     ["relvol_toxhr_0", "rrirpnxm_toxhr_0"],
#     ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
# ]
# interaction_terms_train_df = helper.get_df_with_interaction_terms(train_df, col_pairs)
# interaction_terms_test_df = helper.get_df_with_interaction_terms(test_df, col_pairs)

### Transform the data if needed

#### Ordinary Least Squares

In [ ]:
ols_regression_model = helper.Regression('OLS')
model_attributes = ols_regression_model.train(train_df, RESPONSE_NAME)
model_attributes

In [ ]:
ols_regression_model.get_average_metrics(TEST_START_DATE, TEST_END_DATE, x_cols)

#### OLS w/ Interacting Terms

In [ ]:
ols_interacting_model = helper.Regression('OLS')
ols_interacting_model.execute(interactingTerms_df, RESPONSE_NAME, interactingTerms_df)
ols_interacting_model.get_metric()

#### LASSO

In [ ]:
lasso_model = helper.Regression('LASSO')
lasso_model.execute(train_df, RESPONSE_NAME, test_df)
lasso_model.get_metric()

### XGBoost



In [ ]:
xgb_model = helper.Regression('XGboost')
xgb_model.execute(train_df, RESPONSE_NAME, test_df)
xgb_model.get_metric()